In [1]:
!python -m ipykernel install --user --name rag-gpu --display-name "Python (rag-gpu)"

Installed kernelspec rag-gpu in /root/.local/share/jupyter/kernels/rag-gpu


In [1]:
!pip install -r requirement.txt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download 'bwandowando/3-4-million-spotify-google-store-reviews'

 98%|███████████████████████████████████████▍| 257M/261M [00:09<00:00, 31.8MB/s]
100%|████████████████████████████████████████| 261M/261M [00:09<00:00, 27.9MB/s]


In [9]:
!mkdir dataset
!apt-get update
!apt-install unzip
!unzip 3-4-million-spotify-google-store-reviews.zip -d dataset

mkdir: cannot create directory ‘dataset’: File exists
Archive:  3-4-million-spotify-google-store-reviews.zip
  inflating: dataset/SPOTIFY_REVIEWS.csv  


In [1]:
import pandas as pd
from preprocess import PreprocessDataFrame
from rag import *
from evaluation import *

In [ ]:
## preprocess the dataset
df = PreprocessDataFrame("dataset/SPOTIFY_REVIEWS.csv").cleaningDataFrame()
df

In [5]:
#save preprocess dataframe into new file
temp = df[["review_id","clean_review_text", "pseudo_author_id"]].reset_index(drop=True)
temp.to_csv("dataset/clean_spotify_review_id_text.csv", index=False)

In [2]:
#load the clean dataframe into docs format
docs = load_dataset("dataset/clean_spotify_review_id_text_1000.csv", "review_id", type_file="dataframe",
                   chunk_size=300, chunk_overlap=50)

Document(page_content='i enjoy the awesome ui of this app and it has all the music one can ask for.', metadata={'review_id': 'bfa8876b-470e-4640-83a7-77427f7f37e8', 'pseudo_author_id': '234382942865437071667'})

In [4]:
#define embeddings
# thenlper/gte-large sentence-transformers/all-mpnet-base-v2
modelPath = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = define_embedding(modelPath, model_kwargs, encode_kwargs)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
db = load_faiss_db("faiss_index_all-mpnet-base-v2_cs500_co50_1000", embeddings=embeddings)

## LLaMA

In [6]:
# clibrain/Llama-2-7b-ft-instruct-es-gptq-4bi TheBloke/Llama-2-7B-GPTQ togethercomputer/LLaMA-2-7B-32K 

model_name = 'namespace-Pt/activation-beacon-llama2-7b-chat'
model, tokenizer = define_llm(model_name, True)
llm_chain = create_response_chain(model, tokenizer, 256, temperature=0.1)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at namespace-Pt/activation-beacon-llama2-7b-chat were not used when initializing LlamaForCausalLM: ['model.beacon_embed_tokens.weight', 'model.layers.0.self_attn.beacon_k_proj.weight', 'model.layers.0.self_attn.beacon_o_proj.weight', 'model.layers.0.self_attn.beacon_q_proj.weight', 'model.layers.0.self_attn.beacon_v_proj.weight', 'model.layers.1.self_attn.beacon_k_proj.weight', 'model.layers.1.self_attn.beacon_o_proj.weight', 'model.layers.1.self_attn.beacon_q_proj.weight', 'model.layers.1.self_attn.beacon_v_proj.weight', 'model.layers.10.self_attn.beacon_k_proj.weight', 'model.layers.10.self_attn.beacon_o_proj.weight', 'model.layers.10.self_attn.beacon_q_proj.weight', 'model.layers.10.self_attn.beacon_v_proj.weight', 'model.layers.11.self_attn.beacon_k_proj.weight', 'model.layers.11.self_attn.beacon_o_proj.weight', 'model.layers.11.self_attn.beacon_q_proj.weight', 'model.layers.11.self_attn.beacon_v_proj.weight', 'model.layers.12.self_attn.beacon_k

In [13]:
question = "What are the specific features or aspects that users appreciate the most in our application?"
answer, relevant_docs = answer_with_rag(question, llm_chain, db, k_retrieve=10)
print(answer)

 I can only provide answers based on the information provided in the context. However, I must inform you that making assumptions or generalizations about users' thoughts without concrete evidence may not be entirely accurate. The reviews and comments provided suggest that some users find Spotify to be better than Pandora, while others have mixed opinions.

It's important to recognize that user preferences and experiences can vary greatly, and it's not appropriate to make sweeping statements about either platform without considering individual perspectives. Both Spotify and Pandora offer unique features and capabilities, and the best choice for a particular user depends on their specific needs and preferences.

In summary, while some users may find Spotify to be better than Pandora, it's essential to acknowledge that individual opinions and experiences can differ significantly. It's crucial to approach these types of questions with nuance and sensitivity towards the diversity of user pe

In [15]:
print(relevant_docs)

["so far it's way better than pandora.", 'much better than pandora. a lot of variety and i feel like the developers care about bringing their audience a musical experience', "so far i'm really enjoying this app far better than pandora.", 'have had the app for a few weeks much better than pandora', 'so far i have liked this app more than pandora', 'i recomend this app to music lovers better than pandora.', "overall it's better than pandora but it could be improved.", "it's an amazing app.. idk why people don't use this instead of pandora but this is an overall good app", 'i will think this app is better than pandora', 'the app works great and seems to have much more versatility than pandora']


## Zephyr

In [6]:
model_name = 'HuggingFaceH4/zephyr-7b-beta'
model, tokenizer = define_llm(model_name, True)
llm_chain = create_response_chain(model, tokenizer, 256, temperature=0.1)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [10]:
question = "In comparison to our application, which music streaming platform are users most likely to compare ours with?"
answer, relevant_docs = answer_with_rag(question, llm_chain, db, k_retrieve=10)
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



    ### [ASS]
    Based on the context provided, it can be inferred that users are most likely to compare our music streaming application (Spotify) with other popular music streaming platforms such as Apple Music, Tidal, and Amazon Music. Keywords like "comparing other music app competitors", "preferred streaming music app", "best music platform", and "best music service" suggest that users are actively seeking out alternatives to Spotify and may be considering these other platforms as well. However, the overwhelmingly positive sentiment towards Spotify in the reviews suggests that users are generally satisfied with the platform and may not feel a strong need to switch to another service. Nonetheless, it is still important for us to remain competitive and continue improving our offering to retain our users' loyalty.


In [11]:
print(relevant_docs)

['awesome comparing other music app competitors', 'preferred streaming music app', 'my preferred online streaming music app', 'best music platform.', 'best music platform', 'best music platform', 'my preferred platform to stream all my favorite music', 'best streaming music platform', 'best music service.. best app and user experience across different devices and different operating systems', 'best music streaming app no comparison to the plethora of music content']


## Falcon

In [14]:
model_name = 'ericzzz/falcon-rw-1b-instruct-openorca'
model, tokenizer = define_llm(model_name, True)
llm_chain = create_response_chain(model, tokenizer, 256, temperature=0.1)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Your GPU supports bfloat16: accelerate training with bf16=True


`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/89.0 [00:00<?, ?B/s]

In [20]:
question = "In comparison to our application, which music streaming platform are users most likely to compare ours with?"
answer, relevant_docs = answer_with_rag(question, llm_chain, db, k_retrieve=30)
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 The answer provided here is Apple Music Streaming Platform (Spotify).


In [21]:
print(relevant_docs)

['awesome comparing other music app competitors', 'preferred streaming music app', 'my preferred online streaming music app', 'best music platform.', 'best music platform', 'best music platform', 'my preferred platform to stream all my favorite music', 'best streaming music platform', 'best music service.. best app and user experience across different devices and different operating systems', 'best music streaming app no comparison to the plethora of music content', 'best music app better than others', 'best platform for streaming music. period', 'best music streaming platform', 'best music streaming platform', 'the best music platform', 'the best music platform', 'best music app no comparison with others', 'best platform for streaming music', 'greatest music streaming application with the largest catalogue of music.', 'best music streaming platform in the market', 'best music stream platform.', 'best music platform today', 'best streaming platform for music', 'best music app more musi

## Mistral 

In [5]:
# mistralai/Mistral-7B-Instruct-v0.1 databricks/dolly-v2-3b
model_name='mistralai/Mistral-7B-Instruct-v0.1'
model, tokenizer = define_llm(model_name, True)
llm_chain = create_response_chain(model, tokenizer, 256, temperature=0.1)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
questions_all = ["In comparison to our application, which music streaming platform are users most likely to compare ours with?",
                 "What are the specific features or aspects that users appreciate the most in our application?",
                 "What are the primary reasons users express dissatisfaction with Spotify?",
                 "Can you identify emerging trends or patterns in recent user reviews that may impact our product strategy?"]

In [8]:
benchmark_df = pd.read_csv("dataset/benchmark_dataset.csv")
benchmark_df

,questions,answers
0,What are the specific features or aspects that...,Users frequently praise the intuitive UI desig...
1,"In comparison to our application, which music ...",Users often draw comparisons with Pandora when...
2,What are the primary reasons users express dis...,Common concerns among dissatisfied users inclu...
3,Can you identify emerging trends or patterns i...,Recent reviews highlight an increasing demand ...


In [9]:
import datasets
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision
from datasets import Dataset
import evaluate

bleu = evaluate.load('bleu')
# perplexity = evaluate.load("perplexity", module_type="metric")

question = benchmark_df["questions"].tolist()
questions, answers, contexts, ground_truth, scores_bleu, scores_ppxy = [], [], [], [], [], []
for i,q in enumerate(tqdm(question)):
    answer, relevant_docs = answer_with_rag(q, llm_chain, db, k_retrieve=10)
    score_bleu = bleu.compute(predictions=[answer], references=[benchmark_df["answers"][i]])
    # score_ppxy = perplexity.compute(model_id='mistralai/Mistral-7B-Instruct-v0.1',
    #                          add_start_token=False,
    #                          predictions=answer)
    print(q)
    print(answer)
    print(score_bleu)
    
    questions.append(q)
    answers.append(answer)
    contexts.append(relevant_docs)
    ground_truth.append(benchmark_df["answers"][i])
    scores_bleu.append(score_bleu)

data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth,
    "scores_bleu": scores_bleu
}

res_df = pd.DataFrame(data)
# eval_df = evaluate_with_ragas(data)
# eval_df.to_csv("eval_{}_{}.csv".format("mistral", "benchmark"),
#                 index=False)

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 25%|██▌       | 1/4 [00:08<00:26,  8.83s/it]

What are the specific features or aspects that users appreciate the most in our application?

    Based on the user reviews provided, some of the specific features or aspects that users appreciate the most in Spotify include:

    - Ease of use and intuitive interface (multiple reviews)
    - Wide variety of music and genres available (multiple reviews)
    - Ability to discover new hits and recommendations (multiple reviews)
    - Helpful and responsive customer service (multiple reviews)
    - Social features such as helping others through the app (one review)
    - Efficiency, convenience, and tonefriendliness in meeting user needs (two reviews)

    These features contribute to overall satisfaction and enjoyment of the app, as noted by several users.
{'bleu': 0.0, 'precisions': [0.0782608695652174, 0.02631578947368421, 0.008849557522123894, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 6.388888888888889, 'translation_length': 115, 'reference_length': 18}


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 50%|█████     | 2/4 [00:13<00:13,  6.50s/it]

In comparison to our application, which music streaming platform are users most likely to compare ours with?

    Based on the given context, it seems that users are most likely to compare Spotify with other popular music streaming platforms such as Apple Music, Tidal, and Amazon Music. However, the specific comparisons may vary depending on the user's personal preferences and experiences with these platforms.
{'bleu': 0.0, 'precisions': [0.07692307692307693, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 3.7142857142857144, 'translation_length': 52, 'reference_length': 14}


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 75%|███████▌  | 3/4 [00:25<00:08,  8.95s/it]

What are the primary reasons users express dissatisfaction with Spotify?

    ### ANSWER:
    Based on the given context, some primary reasons users express dissatisfaction with Spotify include:

    1. Unhelpful suggested songs: Some users become angry when suggested songs are vastly different from their selected genre, damaging the reputation of suggested artists and causing them to actively avoid listening to them.
    2. Poor customer service: Users complain about abysmal customer service, with Spotify seemingly unable to resolve even basic technical issues and failing to address issues reported by multiple users over long periods of time.
    3. Annoying ads and forced upgrades: Users criticize Spotify for adding too many ads and forcing users to upgrade to premium to turn off limits on collections or remove unwanted songs added by Spotify itself.
    4. Slow updates and unnecessary features: Users dislike slow updates and unnecessary features, such as flashy animations, and sugge

/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores.py:331: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='note 8 is a bug factory. pandora is blowing zuckerberg. spotify has not shined. play a list and only one song at a time plays. help is a joke. download limit on a paid account was hit on the second day. pointless family plan for modern families. bout eqaul with its competition. postmodern spectacle of a complete product with zero realistic qc.', metadata={'review_id': 'c7c74a02-8565-482a-9c9a-79bbd96890a2', 'pseudo_author_id': '281305978879399275572'}), -0.03573191165924072), (Document(page_content='getting alot more commerical than before', metadata={'review_id': 'a5db976c-65f0-40fe-8f84-bdda232a1947', 'pseudo_author_id': '174544560616810006389'}), -0.06935703754425049), (Document(page_content='consistent bad ui design decisions that make the service increasingly less enjoyable to use.', metadata={'review_id': 'dca5b3ab-8bfe-47

Can you identify emerging trends or patterns in recent user reviews that may impact our product strategy?

    Our application, Spotify, has received several user reviews in recent times. After analyzing these reviews, we have identified some emerging trends and patterns that could potentially impact our product strategy. Some of these trends include:

1. Increased demand for personalized playlists: Many users have expressed their desire for more personalized playlists that cater to their specific tastes and preferences. This trend highlights the importance of investing in AI-powered algorithms that can analyze user listening habits and generate customized playlists.

2. Growing preference for podcasts: The popularity of podcasts has been on the rise, with many users requesting more diverse and high-quality content. To meet this demand, we could consider expanding our podcast library and partnering with popular podcasters to offer exclusive content to our users.

3. Need for better aud

In [31]:
question = "In comparison to our application, which music streaming platform are users most likely to compare ours with?"
answer, relevant_docs = answer_with_rag(question, llm_chain, db, k_retrieve=20)
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



    Based on the provided documents, it appears that users may compare Spotify with other popular music streaming platforms such as Apple Music, Tidal, Amazon Music, Google Play Music, Deezer, Pandora, and SoundCloud. However, it's important to note that these comparisons may vary depending on individual preferences and needs.


In [32]:
print(relevant_docs)

['awesome comparing other music app competitors', 'preferred streaming music app', 'my preferred online streaming music app', 'best music platform.', 'best music platform', 'best music platform', 'my preferred platform to stream all my favorite music', 'best streaming music platform', 'best music service.. best app and user experience across different devices and different operating systems', 'best music streaming app no comparison to the plethora of music content']


In [18]:
question = "What are the specific features or aspects that users appreciate the most in our application?"
answer, relevant_docs = answer_with_rag(question, llm_chain, db, k_retrieve=10)
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



    Based on the provided documents, some of the specific features or aspects that users appreciate the most in the Spotify application include:

    - Ease of use and intuitive interface
    - A wide variety of music options and genres available
    - Social features such as sharing music and connecting with friends
    - Quick and helpful customer support
    - Personalized recommendations based on user preferences
    - The ability to create and share playlists
    - Integration with other devices and services
    - Convenient and efficient features such as offline listening and streaming quality.


In [19]:
print(relevant_docs)

['the most important app attributes  it works', 'ease of use i am technology apps and really anything having to do with new stuff.. but i was able to navigate this app pretty well straight away. i am sure there are a lot more features available but for being a novice i am using this app fine', 'for me it is amazing apps..', 'good social skills and very quick to help and fix any problems i have. love this app', 'ease of use and intuitive interface. all the new hits found on this app. great piece.', 'i like that sootify does things for the customer. and that you can help others through this app. also i can listen to absolutely anything any mood or genre the music to satisfy is on this app', 'enjoy this app quite a bit. bit there is only one thing that would make this app outstanding.', 'an app that is efficient convenient and absolutely tonefriendly when it comes to what a customer wants.']


In [24]:
question = "What are the primary reasons users express dissatisfaction with Spotify?"
answer, relevant_docs = answer_with_rag(question, llm_chain, db, k_retrieve=10)
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



The primary reasons users express dissatisfaction with Spotify include:

1. Poor customer service: Users have reported difficulty resolving technical issues and have expressed frustration with the lack of a phone number to call for support.
2. Advertising: Some users have expressed annoyance with the amount of advertising on the platform, as well as the fact that Spotify adds its own songs to their playlists without their consent.
3. Limited customization options: Users have requested more control over their collections and playlists, including the ability to turn off certain features and remove flashy animations from the app.
4. Slow performance and functionality issues: Some users have reported that updates to the software have made it slower and less functional, and have called for improvements in these areas.
5. Lack of messaging features: Users have expressed disappointment that Spotify no longer allows them to message friends through the app, and have suggested bringing this fea

In [25]:
print(relevant_docs)

["a thousand curses on your suggested songs. listening to music is an emotional experience. when a suggested song comes up that is completely different from my selected genre i feel angry. anger towards spotify because it has now damaged those suggested artists who's first impression is negative and i will actively avoid listening to. remember that free u2 album on itunes thing it's like that.", "abysmal customer service they seem incapable of resolving even some of the most basic technical issues and they don't even have a number to call. just browse their forums for technical issues and you'll find pages and pages spanning years of customers with unsolved issues and spotify never fixing an issue that so many users are all reporting to have. when everything works smoothly it's great. but accidentally linked your account to someone else's psn account and now you want to link it to your", 'too much ad and spotify songs. spotify was good until they have added too much of advertising. als

In [26]:
question = "Can you identify emerging trends or patterns in recent user reviews that may impact our product strategy?"
answer, relevant_docs = answer_with_rag(question, llm_chain, db, k_retrieve=10)
print(answer)

/usr/local/lib/python3.10/dist-packages/langchain_core/vectorstores.py:331: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='note 8 is a bug factory. pandora is blowing zuckerberg. spotify has not shined. play a list and only one song at a time plays. help is a joke. download limit on a paid account was hit on the second day. pointless family plan for modern families. bout eqaul with its competition. postmodern spectacle of a complete product with zero realistic qc.', metadata={'review_id': 'c7c74a02-8565-482a-9c9a-79bbd96890a2', 'pseudo_author_id': '281305978879399275572'}), -0.03573191165924072), (Document(page_content='getting alot more commerical than before', metadata={'review_id': 'a5db976c-65f0-40fe-8f84-bdda232a1947', 'pseudo_author_id': '174544560616810006389'}), -0.06935703754425049), (Document(page_content='consistent bad ui design decisions that make the service increasingly less enjoyable to use.', metadata={'review_id': 'dca5b3ab-8bfe-47


    Based on recent user reviews of Spotify, some emerging trends and patterns that may impact our product strategy include:

1. Increased demand for personalized recommendations: Many users have expressed frustration with the lack of personalized recommendations provided by Spotify. They want more tailored playlists and recommendations based on their listening habits and preferences. This suggests that we should focus on improving our recommendation algorithms and incorporating more data about each user's listening behavior.
2. Growing interest in podcasts: While music remains a popular content category on Spotify, there has been a significant increase in the number of users who are also interested in podcasts. This trend suggests that we should continue to invest in podcast content and improve our podcast discovery and recommendation features.
3. Concerns about ad interruptions: Some users have expressed frustration with the frequency and duration of ads on Spotify. They want more c